In [4]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

BERT_PATH = "../input/huggingface-bert-variants/distilbert-base-uncased/distilbert-base-uncased"

data_dir = Path('../raw_data/AI4Code')

In [5]:
NUM_TRAIN = 200


def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
        .assign(id=path.stem)
        .rename_axis('cell_id')
    )


paths_train = list((data_dir / 'train_data').glob('*.json'))[:NUM_TRAIN]
notebooks_train = [
    read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
]
df = (
    pd.concat(notebooks_train)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)

df

Train NBs:   0%|          | 0/200 [00:00<?, ?it/s]/var/folders/04/sd763rcx6rd1bxvl33_fm1s80000gn/T/ipykernel_81705/2131225060.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pd.read_json(
/var/folders/04/sd763rcx6rd1bxvl33_fm1s80000gn/T/ipykernel_81705/2131225060.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pd.read_json(
/var/folders/04/sd763rcx6rd1bxvl33_fm1s80000gn/T/ipykernel_81705/2131225060.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing 

cell_type                                                                                                                   source
id             cell_id                                                                                                                                    
005a0b5efe318c ad61a02e      code  # Statistics\nimport pandas as pd\nimport numpy as np\nimport math as mt\n\n# Data Visualization\nimport matplotlib....
               6d898218      code  import os\nkaggle_project = 'seattle'\n# Import dataset from local directory './data' or from Kaggle\ndata_dir = ('....
               76a6e832      code  def generate_listings_kfold():\n    # Mark the train dataset with kfold = 5\n    listings = pd.read_csv(f'{data_dir}...
               bdcaaf32      code  # After assigning kfold\n# If error, run the above function then re-load listings_kfold.csv\nlistings.loc[:, ['id', ...
               d3ba2117      code  # Define sheet id and base url\nsheet_id = "1M_qah-ym6O8vDcSmoKAP-lbZRPHUey83R_DJaW3LXfs"\nbase_url = f"https://docs...
...                           ...                                                                                                                      ...
fce6499e20976f 928e14f6  markdown                                                                                                ### Moving Average Method
               11d78dc1  markdown                                          ### Data Preprocessing\n#### For our timeseries model we'll use cities dataset.
               a75031b7  markdown                                                                                                     ### Model Evaluation
               f41423a3  markdown                 #### The above error rate indicates that on any given day there will be a (+/-) 2.39 difference in temp.
               187f9cd5  markdown  #### year has a 12 month cycle therefore we are going to shift the temp by 12 places and then calculate the temp dif...

[8682 rows x 2 columns]

In [6]:
nb_id = df.index.unique('id')[6]
print('Notebook:', nb_id)

print("The disordered notebook:")
nb = df.loc[nb_id, :]
display(nb)
print()

Notebook: 0cf829b9d4e933
The disordered notebook:


,cell_type,source
cell_id,,
44d682f1,code,"import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns\nimport time, os, warnings\ncolor = sns.c..."
29bddfce,code,inspections.head()
5dc02636,code,inspections.describe()
df0f0661,code,inspections.info()
815fae86,code,"fig, ax = plt.subplots(2, 1, figsize = (10, 8))\nsns.boxplot(inspections[""score""], ax = ax[0])\nax[0].set_title(""Bo..."
...,...,...
7b0856e7,markdown,# Inspections
fb06fbbb,markdown,### Pe Descrition vs Grade
b3641fb1,markdown,### Facility Zip vs Grade


In [11]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
)  # Split the string representation of cell_ids into a list
df_orders['cell_order'] = df_orders['cell_order'].str.split()

In [16]:
cell_order = df_orders.loc[nb_id]

print("The ordered notebook:")
nb.loc[cell_order, :]

The ordered notebook:


TypeError: unhashable type: 'list'

In [15]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

cell_ranks = get_ranks(cell_order, list(nb.index))
nb.insert(0, 'rank', cell_ranks)

nb

TypeError: 'Index' object is not callable